##### https://www.analyticsvidhya.com/blog/2022/04/building-vehicle-counter-system-using-opencv/

Object tracking with Centroid Tracking Algorithm

In [23]:
import cv2
import numpy as np

tracker = EuclideanDistTracker()
cap = cv2.VideoCapture('../images/roads-1952.mp4')
_, frame1 = cap.read()
_, frame2 = cap.read()

while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    height, width = blur.shape
    _, threshold = cv2.threshold(blur, 23, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(threshold, (1,1), iterations=1)
    contours,_, = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
        #if cv2.contourArea(contour) >= 300:
        #    cv2.rectangle(diff, (x,y), (x+w,y+h), (0,255,255), 3)
        if cv2.contourArea(contour) <300:
                continue
        detections.append([x,y,w,h])
    boxes_ids = tracker.update(detections)
    for box_ids in boxes_ids:
        x,y,w,h,id = box_ids
        cv2.putText(frame1, str(id), (x,y-15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
        cv2.rectangle(frame1, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.imshow('frame',frame1)

    frame1 = frame2
    ret,frame2 = cap.read()
    key = cv2.waitKey(30)
    if key == ord('q'):
        break

cv2.destroyAllWindows()

In [22]:
import math

class EuclideanDistTracker:
    def __init__(self):
        self.center_points = {}
        self.id_count = 0
        
    def update(self,objects_rect):
        objects_bbs_ids = []
        for rect in objects_rect:
            x, y, w, h = rect
            center_x = (x+x+w)//2
            center_y = (y + y + h) // 2
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(center_x - pt[0], center_y - pt[1])
                if dist < 25:
                    self.center_points[id] = (center_x, center_y)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break
            if same_object_detected is False:
                self.center_points[self.id_count] = (center_x, center_y)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1
        new_center_points = {}    
        for obj_bb_id in objects_bbs_ids:
            var,var,var,var, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center
        self.center_points = new_center_points.copy()
        return objects_bbs_ids